<a href="https://colab.research.google.com/github/heinhtet14/ML_ZoomCamp_Cohort_2025/blob/main/ML_ZoomCamp_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

### Question 1

What is the most frequent observation (mode) for the column `industry`?

- `NA`
- `technology`
- `healthcare`
- `retail`

In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [24]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [25]:
df.dtypes

,0
lead_source,object
industry,object
number_of_courses_viewed,int64
annual_income,float64
employment_status,object
location,object
interaction_count,int64
lead_score,float64
converted,int64


In [26]:
df.isna().sum()

,0
lead_source,128
industry,134
number_of_courses_viewed,0
annual_income,181
employment_status,100
location,63
interaction_count,0
lead_score,0
converted,0


In [27]:
categorical = df.select_dtypes(include='object').columns
df[categorical] = df[categorical].fillna('na')


In [28]:
numerical = df.select_dtypes(include=['int64', 'float64']).columns
df[numerical] = df[numerical].fillna(0.0)

In [29]:
df.isna().sum()

,0
lead_source,0
industry,0
number_of_courses_viewed,0
annual_income,0
employment_status,0
location,0
interaction_count,0
lead_score,0
converted,0


In [30]:
df['industry'].mode()[0]

'retail'

### Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset.
In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

- `interaction_count` and `lead_score`
- `number_of_courses_viewed` and `lead_score`
- `number_of_courses_viewed` and `interaction_count`
- `annual_income` and `interaction_count`

Only consider the pairs above when answering this question.

### Split the data

- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
- Make sure that the target value `converted` is not in your dataframe.

In [31]:
corr_matrix = df[numerical].corr()
corr_matrix.style.background_gradient(cmap='plasma')

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [32]:
# Extract the relevant pairs and unstack
corr_matrix_unstacked = corr_matrix.unstack()

# Sort the pairs by absolute correlation value in descending order
sorted_pairs = corr_matrix_unstacked.sort_values(key=abs, ascending=False)

# Filter for the relevant pairs specified in the question, excluding self-correlations and duplicates
relevant_pairs = [
    ("interaction_count", "lead_score"),
    ("number_of_courses_viewed", "lead_score"),
    ("number_of_courses_viewed", "interaction_count"),
    ("annual_income", "interaction_count")
]

print("Correlation values for the specified pairs:")
for feature1, feature2 in relevant_pairs:
    try:
        correlation_value = corr_matrix_unstacked.loc[feature1, feature2]
        print(f"  {feature1} and {feature2}: {correlation_value:.4f}")
    except KeyError:
        # Handle cases where the pair might be in reverse order in the unstacked series
        correlation_value = corr_matrix_unstacked.loc[feature2, feature1]
        print(f"  {feature1} and {feature2}: {correlation_value:.4f}")

# Find the pair with the biggest correlation among the relevant pairs
relevant_correlations = {f"{f1} and {f2}": corr_matrix_unstacked.get((f1, f2), corr_matrix_unstacked.get((f2, f1)))
                         for f1, f2 in relevant_pairs}

highest_correlation_pair = max(relevant_correlations.items(), key=lambda x: abs(x[1]))

print("\nThe two features with the biggest correlation among the specified pairs are:", highest_correlation_pair[0])
print("Correlation value:", highest_correlation_pair[1])

Correlation values for the specified pairs:
  interaction_count and lead_score: 0.0099
  number_of_courses_viewed and lead_score: -0.0049
  number_of_courses_viewed and interaction_count: -0.0236
  annual_income and interaction_count: 0.0270

The two features with the biggest correlation among the specified pairs are: annual_income and interaction_count
Correlation value: 0.02703647240481443


In [33]:
corr_matrix_unstacked

number_of_courses_viewed  number_of_courses_viewed    1.000000
                          annual_income               0.009770
                          interaction_count          -0.023565
                          lead_score                 -0.004879
                          converted                   0.435914
annual_income             number_of_courses_viewed    0.009770
                          annual_income               1.000000
                          interaction_count           0.027036
                          lead_score                  0.015610
                          converted                   0.053131
interaction_count         number_of_courses_viewed   -0.023565
                          annual_income               0.027036
                          interaction_count           1.000000
                          lead_score                  0.009888
                          converted                   0.374573
lead_score                number_of_courses_viewed   -0.004879
                          annual_income               0.015610
                          interaction_count           0.009888
                          lead_score                  1.000000
                          converted                   0.193673
converted                 number_of_courses_viewed    0.435914
                          annual_income               0.053131
                          interaction_count           0.374573
                          lead_score                  0.193673
                          converted                   1.000000
dtype: float64

In [34]:
# Split the Data
from sklearn.model_selection import train_test_split

In [35]:
# Split the data into 80% full train and 20% test
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

# Split the full train into 60% train and 20% validation
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)  # 0.25 * 80% = 20%

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

# Verify the sizes
print(f"Train size: {len(df_train)}")
print(f"Validation size: {len(df_val)}")
print(f"Test size: {len(df_test)}")

Train size: 876
Validation size: 293
Test size: 293


### Question 3

- Calculate the mutual information score between `converted` and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the biggest mutual information score?

- `industry`
- `location`
- `lead_source`
- `employment_status`


In [36]:
from sklearn.metrics import mutual_info_score

# Select categorical columns from the training set
categorical_columns = df_train.select_dtypes(include=['object']).columns

# Calculate mutual information scores
mi_scores = {}
for col in categorical_columns:
    mi_scores[col] = mutual_info_score(df_train[col], y_train)

# Sort and display the mutual information scores rounded to 2 decimal places
sorted_mi_scores = sorted(mi_scores.items(), key=lambda x: x[1], reverse=True)
for col, score in sorted_mi_scores:
    print(f"Mutual Information between {col} and target: {round(score, 2)}")

Mutual Information between lead_source and target: 0.03
Mutual Information between employment_status and target: 0.01
Mutual Information between industry and target: 0.01
Mutual Information between location and target: 0.0


### Question 4

- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
  - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
  - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.64
- 0.74
- 0.84
- 0.94

In [38]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
accuracy_score(y_val, y_pred)

0.7303754266211604

In [39]:
df_train.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score'],
      dtype='object')

### Question 5

- Let's find the least useful feature using the _feature elimination_ technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

- `'industry'`
- `'employment_status'`
- `'lead_score'`

> **Note**: The difference doesn't have to be positive.

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define the features and target
features = ['industry',
            'employment_status', 'lead_score']
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
baseline_accuracy = accuracy_score(y_val, y_pred)

# Perform feature elimination
accuracy_differences = {}
for feature in features:
    # Exclude the feature

    train_dict = df_train.drop(columns=[feature]).to_dict(orient='records')
    X_train_subset= dv.fit_transform(train_dict)

    val_dict = df_val.drop(columns=[feature]).to_dict(orient='records')
    X_val_subset = dv.transform(val_dict)

    # Train the model without the feature
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_subset, y_train)
    y_pred = model.predict(X_val_subset)
    accuracy = accuracy_score(y_val, y_pred)

    # Calculate the difference
    accuracy_differences[feature] = baseline_accuracy - accuracy

# Find the feature with the smallest difference
least_useful_feature = min(accuracy_differences, key=accuracy_differences.get)
print("Feature with the smallest difference:", least_useful_feature)
print("Accuracy differences:", accuracy_differences)

Feature with the smallest difference: industry
Accuracy differences: {'industry': -0.010238907849829393, 'employment_status': 0.0, 'lead_score': 0.0}


### Question 6

- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter `C`: `[0.01, 0.1, 1, 10, 100]`.
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these `C` leads to the best accuracy on the validation set?

- 0.01
- 0.1
- 1
- 10
- 100

> **Note**: If there are multiple options, select the smallest `C`.


In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define the features and target
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

# Define the values of C to try
C_values = [0.01, 0.1, 1, 10, 100]

# Train models and calculate accuracy
accuracies = {}
for C in C_values:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    accuracies[C] = round(accuracy, 4)

# Find the best C value
best_C = max(accuracies, key=accuracies.get)
print("Accuracies for different C values:")
for C, acc in accuracies.items():
    print(f"C={C}, Validation Accuracy={acc}")
print(f"\nBest C value: {best_C} with Validation Accuracy: {accuracies[best_C]}")

Accuracies for different C values:
C=0.01, Validation Accuracy=0.7304
C=0.1, Validation Accuracy=0.7304
C=1, Validation Accuracy=0.7304
C=10, Validation Accuracy=0.7304
C=100, Validation Accuracy=0.7304

Best C value: 0.01 with Validation Accuracy: 0.7304
